In [ ]:
import os
import sys
# If we don't need CUDA, do this before importing TF
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
import pandas as pd
import tqdm
import tqdm.notebook
import scipy.stats

import matplotlib.pyplot as plt

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_visible_devices([gpus[1]], 'GPU')

sys.path.append("/nfs/scistore12/chlgrp/vvolhejn/smooth")

os.chdir("/nfs/scistore12/chlgrp/vvolhejn/smooth/logs/0224_gp1/")

In [ ]:
%load_ext autoreload
%aimport smooth.datasets
%aimport smooth.model
%aimport smooth.analysis
%aimport smooth.callbacks
%aimport smooth.measures
%aimport smooth.util
%autoreload 1

In [ ]:
df = pd.read_feather("measures.feather")
smooth.analysis.remove_constant_columns(df)
df = smooth.analysis.expand_dataset_columns(df)
df["log_dir"] = df["log_dir"].str.split("/").str.get(-1)
df.head()

In [ ]:
for col in ["actual_epochs", "train_loss", "test_loss"]:
    if df[col].dtype == "object":
        continue
    plt.hist(df[col], bins=20)
    plt.title(col)
    plt.show()

In [ ]:
df1 = df[(df["seed"] == 1) & (df["lengthscale"] == 0.1) & (df["train_loss"] < 10)]
plt.scatter(df1["samples_train"], df1["seg_total_variation"], alpha=0.3)

## Results

These models are larger and so I was hoping they could successfully fit datasets with `lengthscale == 0.1`.
This is not the case, however, as we can see from the plots:

In [ ]:
df1 = df[(df["seed"] == 4) & (df["lengthscale"] == 0.3) & (df["train_loss"] < 0.01)]
# df1 = df1[(df1[""])]

# plt.scatter(df1["samples_train"], df1["seg_total_variation"], alpha=0.3)
df1 = df1[df1["samples_train"] == 100]
print(len(df1))

dataset = smooth.datasets.GaussianProcessDataset.from_name(df1.iloc[0]["dataset"])
x = dataset.x_test

ax = plt.subplot()
ax.plot(x, dataset.y_test, color="C0")

for i, row in list(df1.iterrows()):
    log_dir = row["log_dir"]
    model = tf.keras.models.load_model(os.path.join(log_dir, "model.h5"))
    y = model.predict(x)
    color = {
        10: "C1",
        30: "C2",
        100: "C3",
    }[row["samples_train"]]
    ax.plot(x, y, alpha=0.1, color=color)


#     smooth.analysis.plot_shallow(model, dataset, title=log_dir)

## Which hyperparameters work the best across datasets?

For each dataset, find which hyperparameters were the most successful in terms of training loss.

In [ ]:
hps = df.sort_values("train_loss").groupby("dataset").first()[["init_scale", "learning_rate"]]
# hps["hp"] = hp
groups0 = hps.groupby(["init_scale", "learning_rate"], as_index=False)
groups = {}
for hp, g_df in groups0:
    groups[hp] = len(g_df)

print("Which hyperparameters are the best?")
groups
# .first()